In [ ]:
!pip install open3d
!pip install trimesh

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 447.7/447.7 MB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.9/7.9 MB 100.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.8/139.8 kB 11.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 69.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 63.7 MB/s eta 0:00:00
  Attempting uninstall: widgetsnbextension
    Found existing installation: widgetsnbextension 3.6.10
    Uninstalling widgetsnbextension-3.6.10:
      Successfully uninstalled widgetsnbextension-3.6.10
  Attempting uninstall: ipywidgets
    Found existing installation: ipywidgets 7.7.1
    Uninstalling ipywidgets-7.7.1:
      Successfully uninstalled ipywidgets-7.7.1
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 709.0/709.0 kB 12.3 MB/s eta 0:00:00


In [ ]:
import pandas as pd
import zipfile
import os
from google.colab import drive
import zipfile
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
#import open3d as o3d
import trimesh
import numpy as np
import random
import gc
from torch.utils.data import TensorDataset, DataLoader

## Loading Dataset

In [ ]:
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [ ]:
print(os.listdir('/content/drive/My Drive'))

['FINANCIAL LITERACY CERTIFICATE.pdf', 'KUZAYET BMC.pdf', 'GST 208 RECEIPT.pdf', 'VID-20240508-WA0014.mp4', 'BAGAI GLORY RESUME.pdf', 'BAGAI GLORY HEADSHOT.jpeg', '300LVL 2ND SEMESTER', 'project proposal template', 'RENUE DOCUMENT', 'Colab Notebooks', 'archive (1).zip', 'archive.zip']


In [ ]:
zip_path = '/content/drive/My Drive/archive.zip'


In [ ]:
extract_path = '/content/ModelNet10'
with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(extract_path)


In [ ]:
if os.path.exists('/content/ModelNet10'):
    print(os.listdir('/content/ModelNet10'))
else:
    print("Directory does not exist. Please extract the dataset again.")

['metadata_modelnet10.csv', 'ModelNet10']


In [ ]:
root_dir = '/content/ModelNet10'
modelnet_dir = os.path.join(root_dir, 'ModelNet10')
print(os.listdir(modelnet_dir))


['desk', 'bathtub', 'toilet', 'sofa', 'table', 'bed', 'night_stand', 'chair', 'monitor', 'dresser']


In [ ]:
# List the categories available
categories = os.listdir(os.path.join(extract_path, 'ModelNet10'))
print("Available categories:", categories)

Available categories: ['desk', 'bathtub', 'toilet', 'sofa', 'table', 'bed', 'night_stand', 'chair', 'monitor', 'dresser']


In [ ]:
file_path = '/content/ModelNet10/ModelNet10/chair/train/chair_0396.off'
mesh = trimesh.load(file_path)

In [ ]:
# Convert Mesh to Voxel Grid (voxel size determines resolution)
voxel_grid = mesh.voxelized(pitch=0.05)  # pitch is voxel size

In [ ]:
# Convert voxel grid to dense boolean numpy array
voxel_matrix = voxel_grid.matrix

In [ ]:
# Visualization Function for Voxel Grids
def visualize_voxel(voxel_data):
    fig = plt.figure()
    ax = fig.add_subplot(111, projection='3d')
    ax.voxels(voxel_data, edgecolor='k')
    plt.show()


In [ ]:
# Visualize the voxel grid
#visualize_voxel(voxel_matrix)

## Dataset Preparation

In [ ]:
# Normalize voxel grids to shape (32, 32, 32).


def load_voxel(file_path, pitch=0.05, target_shape=(32, 32, 32)):
    mesh = trimesh.load(file_path)
    voxel_grid = mesh.voxelized(pitch=pitch)
    voxel_data = voxel_grid.matrix.astype(np.float32)

    # Resize/Crop to target_shape if necessary (simplified)
    if voxel_data.shape != target_shape:
        padded = np.zeros(target_shape, dtype=np.float32)
        min_shape = np.minimum(voxel_data.shape, target_shape)
        padded[:min_shape[0], :min_shape[1], :min_shape[2]] = voxel_data[:min_shape[0], :min_shape[1], :min_shape[2]]
        voxel_data = padded

    return voxel_data

In [ ]:
# adding noise to the data

def add_noise(voxel_data, noise_factor=0.2):
    noisy = voxel_data.copy()
    noise = np.random.binomial(1, noise_factor, size=voxel_data.shape)
    noisy = np.clip(noisy + noise, 0, 1)
    return noisy

In [ ]:
# Prepare Dataset Batch
import random
dataset_dir = '/content/ModelNet10/ModelNet10/chair/train'
files = [f for f in os.listdir(dataset_dir) if f.endswith('.off')]
random.shuffle(files)

X_train = []
Y_train = []

In [ ]:
from torch.utils.data import Dataset, DataLoader

class VoxelDataset(Dataset):
    def __init__(self, file_paths, dataset_dir, transform=None):
        self.file_paths = file_paths
        self.dataset_dir = dataset_dir
        self.transform = transform

    def __len__(self):
        return len(self.file_paths)

    def __getitem__(self, idx):
        file_name = self.file_paths[idx]
        file_path = os.path.join(self.dataset_dir, file_name)
        voxel = load_voxel(file_path)  # Load .off file and convert to voxel grid

        noisy_voxel = add_noise(voxel)  # Add noise to voxel grid

        # Convert to tensor and permute to (Channels, D, H, W)
        noisy_voxel = torch.tensor(noisy_voxel, dtype=torch.float32).unsqueeze(0) # Add channel dimension
        voxel = torch.tensor(voxel, dtype=torch.float32).unsqueeze(0) # Add channel dimension


        return noisy_voxel, voxel

In [ ]:
# Prepare Dataset & DataLoader
dataset = VoxelDataset(files[:10], dataset_dir)  # Load only 10 samples for demo
dataloader = DataLoader(dataset, batch_size=1, shuffle=True)

**Build Denoising Autoencoder (3D Conv Model)**

In [ ]:
import torch.nn as nn

class DenoisingAutoencoder3D(nn.Module):
    def __init__(self):
        super(DenoisingAutoencoder3D, self).__init__()

        # Encoder
        self.encoder = nn.Sequential(
            nn.Conv3d(1, 32, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool3d(2, stride=2),  # (16x16x16)
            nn.Conv3d(32, 64, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool3d(2, stride=2),  # (8x8x8)
        )

        # Decoder
        self.decoder = nn.Sequential(
            nn.Conv3d(64, 64, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.Upsample(scale_factor=2, mode='trilinear', align_corners=True),  # (16x16x16)
            nn.Conv3d(64, 32, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.Upsample(scale_factor=2, mode='trilinear', align_corners=True),  # (32x32x32)
            nn.Conv3d(32, 1, kernel_size=3, padding=1),
            nn.Sigmoid()
        )

    def forward(self, x):
        x = self.encoder(x)
        x = self.decoder(x)
        return x

In [ ]:
# Instantiate model
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = DenoisingAutoencoder3D().to(device)

In [ ]:
# Loss and Optimizer
criterion = nn.BCELoss()  # Binary Cross Entropy Loss for voxel data
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)


**Training the AutoEncoder**

In [ ]:
# Training Loop
epochs = 20

for epoch in range(epochs):
    total_loss = 0
    for batch in dataloader:
        noisy_voxels, clean_voxels = batch
        noisy_voxels = noisy_voxels.to(device)
        clean_voxels = clean_voxels.to(device)

        # Forward pass
        outputs = model(noisy_voxels)
        loss = criterion(outputs, clean_voxels)

        # Backward pass and optimization
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        total_loss += loss.item()

    print(f"Epoch [{epoch+1}/{epochs}], Loss: {total_loss/len(dataloader):.4f}")


Evaluating and Visualising the reconstruction

In [ ]:
# Pick a random test sample
index = random.randint(0, len(X_train_tensor)-1)
test_sample = X_train_tensor[index:index+1].to(device)
ground_truth = Y_train_tensor[index:index+1].to(device)

In [ ]:
# Get model prediction
model.eval()
with torch.no_grad():
    reconstructed = model(test_sample)

In [ ]:
# Convert tensors to numpy for visualization
test_sample_np = test_sample.cpu().numpy()[0, 0]
reconstructed_np = reconstructed.cpu().numpy()[0, 0]
ground_truth_np = ground_truth.cpu().numpy()[0, 0]

In [ ]:
# Visualize Middle Slice
def show_slice(voxel, title):
    plt.imshow(voxel[:, :, 16], cmap='gray')
    plt.title(title)
    plt.axis('off')
    plt.show()

In [ ]:
show_slice(test_sample_np, "Noisy Input")
show_slice(reconstructed_np, "Reconstructed Output")
show_slice(ground_truth_np, "Ground Truth")